In [1]:
import numpy as np
import keras
from keras import backend as K
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

from keras.layers import Input, Dense, Activation
from keras.layers import Conv1D, AveragePooling1D, Conv2DTranspose
from keras.layers import Flatten, Reshape, Lambda
from keras.layers import Dropout
from keras.utils import plot_model
from keras import Model

import os
import os.path as osp
import sys
sys.path.append('/home/jcollins')

import ot
import numpy as np

import gc

Using TensorFlow backend.


In [2]:
masses = [200,300,400,500]
signal_data_files = [np.load('/data1/users/jcollins/monosquark_samples/VV/VV+_'+str(mass)+'_12.npz') for mass in masses]

ttbarfiles = ['tt_12_0_' + str(i) + '_0.npz' for i in range(5)]
ttbarfiles.extend(['tt_12_1_' + str(i) + '_0.npz' for i in range(10)])
bg_data_files = [np.load('/data1/users/jcollins/monosquark_samples/ttbar/' + filename) for filename in ttbarfiles]

In [3]:
signal_data = [file['constituents'] for file in signal_data_files]
signal_data = np.concatenate(signal_data, axis=0)
bg_data = [file['constituents'] for file in bg_data_files]
bg_data = np.concatenate(bg_data, axis=0)

np.random.shuffle(signal_data)
np.random.shuffle(bg_data)

In [4]:
print(len(bg_data),len(signal_data))

219285 247098


In [5]:
numtrain = 200000
numvalid = 10000
numtotal = numtrain+numvalid

labels = np.append(np.zeros(numtotal),np.ones(numtotal))
data = np.append(bg_data,signal_data,axis=0)
data[:,:]
perms = np.random.permutation(2*numtotal)
data = data[perms]
for event in data:
    event[:,0] = event[:,0] / np.sum(event[:,0],axis=-1)
labels = labels[perms]

train_data = data[:numtrain*2]
valid_data = data[numtrain*2:]
train_labels = labels[:numtrain*2]
valid_labels = labels[numtrain*2:]

In [6]:
dropprob = 0.1
nodes = 512

numparticles = len(bg_data[0])
lenparticle = len(bg_data[0,0])
inputs = Input(shape=(numparticles,lenparticle),name='input')
layer = Conv1D(nodes,1,activation='relu',name='1_conv1d_0')(inputs)
layer = Dropout(dropprob)(layer)
layer = Conv1D(nodes,1,activation='relu',name='2_conv1d_1')(layer)
layer = Dropout(dropprob)(layer)
layer = Conv1D(nodes,1,activation='relu',name='3_conv1d_2')(layer)
layer = AveragePooling1D(pool_size=numparticles,name='4_avgpool')(layer)
layer = Reshape((nodes,), name='5_reshape')(layer)
layer = Dropout(dropprob)(layer)
layer = Dense(nodes,activation='relu',name='6_dense_0')(layer)
layer = Dropout(dropprob)(layer)
layer = Dense(nodes,activation='relu',name='7_dense_1')(layer)
layer = Dropout(dropprob)(layer)
layer = Dense(nodes,activation='relu',name='8_dense_2')(layer)
layer = Dropout(dropprob)(layer)
output = Dense(1,activation='sigmoid',name='output')(layer)

model = Model(inputs, output)
model.compile(loss=keras.losses.binary_crossentropy, optimizer=keras.optimizers.Adam())

In [11]:
model.fit(train_data,train_labels,epochs=200,batch_size=100,validation_data=(valid_data,valid_labels))

Train on 400000 samples, validate on 20000 samples
Epoch 1/200
400000/400000 [==============================] - 107s 266us/step - loss: 0.6612 - val_loss: 0.6545
Epoch 2/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.6542 - val_loss: 0.6502
Epoch 3/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.6517 - val_loss: 0.6492
Epoch 4/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.6500 - val_loss: 0.6473
Epoch 5/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.6488 - val_loss: 0.6451
Epoch 6/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.6479 - val_loss: 0.6452
Epoch 7/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.6473 - val_loss: 0.6435
Epoch 8/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.6464 - val_loss: 0.6465
Epoch 9/200
400000/400000 [==============================] - 

KeyboardInterrupt: 

In [ ]:
def create_dir(dir_path):
    ''' Creates a directory (or nested directories) if they don't exist.
    '''
    if not osp.exists(dir_path):
        os.makedirs(dir_path)

    return dir_path

save_period = 5
epochs = 200

output_dir = './models/'
experiment_name = 'hardest_jet_1'

train_output_dir = create_dir(osp.join(output_dir, experiment_name))
modelcheckpoint = keras.callbacks.ModelCheckpoint(train_output_dir + '/model_{epoch:02d}-{val_loss:.2e}.hdf5', period = save_period)

callbacks = [modelcheckpoint]

model.fit(train_data,train_labels,epochs=epochs,batch_size=100,validation_data=(valid_data,valid_labels),
         callbacks = callbacks)

Train on 400000 samples, validate on 20000 samples
Epoch 1/200
400000/400000 [==============================] - 106s 265us/step - loss: 0.6321 - val_loss: 0.6238
Epoch 2/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.6281 - val_loss: 0.6210
Epoch 3/200
400000/400000 [==============================] - 107s 268us/step - loss: 0.6245 - val_loss: 0.6228
Epoch 4/200
400000/400000 [==============================] - 107s 268us/step - loss: 0.6221 - val_loss: 0.6077
Epoch 5/200
400000/400000 [==============================] - 107s 268us/step - loss: 0.6187 - val_loss: 0.6111
Epoch 6/200
400000/400000 [==============================] - 107s 269us/step - loss: 0.6183 - val_loss: 0.6126
Epoch 7/200
400000/400000 [==============================] - 108s 269us/step - loss: 0.6157 - val_loss: 0.6020
Epoch 8/200
400000/400000 [==============================] - 108s 269us/step - loss: 0.6141 - val_loss: 0.6092
Epoch 9/200
400000/400000 [==============================] - 

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



400000/400000 [==============================] - 107s 268us/step - loss: 0.6076 - val_loss: 0.5992
Epoch 17/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.5953 - val_loss: 0.5888
Epoch 45/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.5952 - val_loss: 0.5867
Epoch 46/200
400000/400000 [==============================] - 107s 268us/step - loss: 0.5948 - val_loss: 0.5875
Epoch 47/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.5957 - val_loss: 0.5896
Epoch 48/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.5941 - val_loss: 0.5875
Epoch 49/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.5942 - val_loss: 0.5923
Epoch 50/200
400000/400000 [==============================] - 107s 268us/step - loss: 0.5940 - val_loss: 0.5899
Epoch 51/200
400000/400000 [==============================] - 107s 267us/step - loss: 0.5939 - val_loss: 0.5905
Epoch

In [10]:
model.fit(train_data,train_labels,epochs=200,batch_size=1600,validation_data=(valid_data,valid_labels))

Train on 400000 samples, validate on 20000 samples
Epoch 1/200
400000/400000 [==============================] - 97s 243us/step - loss: 0.6665 - val_loss: 0.6630
Epoch 2/200
400000/400000 [==============================] - 97s 242us/step - loss: 0.6642 - val_loss: 0.6600
Epoch 3/200
 16000/400000 [>.............................] - ETA: 1:32 - loss: 0.6582

KeyboardInterrupt: 

In [73]:
model.fit(train_data,train_labels,epochs=200,batch_size=300,validation_data=(valid_data,valid_labels))

Train on 400000 samples, validate on 20000 samples
Epoch 1/200
400000/400000 [==============================] - 106s 264us/step - loss: 0.6678 - val_loss: 0.6627
Epoch 2/200
400000/400000 [==============================] - 105s 263us/step - loss: 0.6616 - val_loss: 0.6589
Epoch 3/200
400000/400000 [==============================] - 105s 263us/step - loss: 0.6585 - val_loss: 0.6552
Epoch 4/200
400000/400000 [==============================] - 105s 263us/step - loss: 0.6539 - val_loss: 0.6517
Epoch 5/200
400000/400000 [==============================] - 105s 263us/step - loss: 0.6505 - val_loss: 0.6474
Epoch 6/200
 91800/400000 [=====>........................] - ETA: 1:19 - loss: 0.6492

KeyboardInterrupt: 

In [71]:
model.fit(train_data,train_labels,epochs=200,batch_size=300,validation_data=(valid_data,valid_labels))

Train on 400000 samples, validate on 20000 samples
Epoch 1/200
400000/400000 [==============================] - 48s 121us/step - loss: 0.6684 - val_loss: 0.6670
Epoch 2/200
400000/400000 [==============================] - 44s 111us/step - loss: 0.6628 - val_loss: 0.6586
Epoch 3/200
400000/400000 [==============================] - 44s 111us/step - loss: 0.6596 - val_loss: 0.6579
Epoch 4/200
400000/400000 [==============================] - 45s 111us/step - loss: 0.6577 - val_loss: 0.6575
Epoch 5/200
400000/400000 [==============================] - 45s 112us/step - loss: 0.6542 - val_loss: 0.6532
Epoch 6/200
400000/400000 [==============================] - 45s 112us/step - loss: 0.6518 - val_loss: 0.6489
Epoch 7/200
400000/400000 [==============================] - 45s 112us/step - loss: 0.6486 - val_loss: 0.6489
Epoch 8/200
400000/400000 [==============================] - 45s 112us/step - loss: 0.6473 - val_loss: 0.6468
Epoch 9/200
400000/400000 [==============================] - 45s 112u

KeyboardInterrupt: 

In [69]:
model.fit(train_data,train_labels,epochs=200,batch_size=300,validation_data=(valid_data,valid_labels))

Train on 400000 samples, validate on 20000 samples
Epoch 1/200
400000/400000 [==============================] - 21s 51us/step - loss: 0.6673 - val_loss: 0.6618
Epoch 2/200
400000/400000 [==============================] - 20s 49us/step - loss: 0.6566 - val_loss: 0.6518
Epoch 3/200
400000/400000 [==============================] - 20s 49us/step - loss: 0.6498 - val_loss: 0.6496
Epoch 4/200
400000/400000 [==============================] - 20s 49us/step - loss: 0.6469 - val_loss: 0.6510
Epoch 5/200
400000/400000 [==============================] - 20s 49us/step - loss: 0.6452 - val_loss: 0.6456
Epoch 6/200
400000/400000 [==============================] - 20s 49us/step - loss: 0.6442 - val_loss: 0.6471
Epoch 7/200
400000/400000 [==============================] - 20s 50us/step - loss: 0.6429 - val_loss: 0.6453
Epoch 8/200
400000/400000 [==============================] - 20s 50us/step - loss: 0.6423 - val_loss: 0.6453
Epoch 9/200
400000/400000 [==============================] - 20s 50us/step - 

KeyboardInterrupt: 

In [65]:
model.fit(train_data,train_labels,epochs=200,batch_size=300,validation_data=(valid_data,valid_labels))

Train on 400000 samples, validate on 20000 samples
Epoch 1/200
400000/400000 [==============================] - 13s 33us/step - loss: 0.6690 - val_loss: 0.6653
Epoch 2/200
400000/400000 [==============================] - 12s 31us/step - loss: 0.6620 - val_loss: 0.6617
Epoch 3/200
400000/400000 [==============================] - 12s 31us/step - loss: 0.6580 - val_loss: 0.6572
Epoch 4/200
400000/400000 [==============================] - 12s 31us/step - loss: 0.6546 - val_loss: 0.6593
Epoch 5/200
400000/400000 [==============================] - 12s 31us/step - loss: 0.6518 - val_loss: 0.6534
Epoch 6/200
400000/400000 [==============================] - 12s 31us/step - loss: 0.6494 - val_loss: 0.6520
Epoch 7/200
400000/400000 [==============================] - 12s 31us/step - loss: 0.6484 - val_loss: 0.6530
Epoch 8/200
400000/400000 [==============================] - 12s 31us/step - loss: 0.6473 - val_loss: 0.6489
Epoch 9/200
400000/400000 [==============================] - 12s 31us/step - 

KeyboardInterrupt: 

In [34]:
train_data[5]

array([[3.43942078e+02, 5.07611036e-01, 1.02473176e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.42019119e+02, 7.79691279e-01, 7.21251488e-01, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.07226105e+01, 1.52828991e-01, 1.67351723e+00, ...,
        1.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [39]:
np.sum(data[0,:,0],axis=-1)

703.7472639679909

In [43]:
data[0]

array([[ 0.22070922,  0.40778416, -2.45515513, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.14192642, -0.61937547, -2.71475029, ...,  0.        ,
         0.        ,  0.        ],
       [ 0.1364755 ,  0.40481746, -2.4508605 , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ]])